### Quantum Entanglement from Vector Similarity: An Entropy-analysis

Date: February 10th, 2026.

*Runtime: 21 min. CPU: 64 cores ($\approx$62.5% peak usage, sustained $\approx$40 cores). RAM: 256 GiB (
$\approx$1.95% usage, $\approx$5 GiB). Network: peak egress $\approx$391 KB/s. GPU: n/a.*

The dynamics of complexity and the distribution of information were analyzed in a 34-logical-qubit quantum system, generated from the projection of classical correlations. A model was implemented that maps the geometric similarity between pairs of random vectors (uniform distribution in $[0, 1)^{1024}$) to probability amplitudes. This mapping uses a parameterized rotation defined by $\theta = (\vec{u} \cdot \vec{v})(\pi - \delta)$ where $\delta$ introduces a uniform stochastic modulation ($\delta \in [0.075, 0.75]$), encoding the qubit state as $|\psi\rangle = \cos\left(\frac{\theta}{2}\right) |0\rangle + \sin\left(\frac{\theta}{2}\right) |1\rangle$.

The system architecture is structured into 17 disjoint pairs, where entanglement is explicitly induced through intra-pair CNOT gates, evolving in a composite Hilbert space of $2^{34}$ dimensions ($\mathcal{H}*{1} \otimes \dots \otimes \mathcal{H}*{34}$). To simulate realistic experimental conditions, a noise model based on the IBM Heron backend topology (via Qiskit Aer) was integrated, adding decoherence and gate errors to the intrinsic parametric uncertainty.

Analysis of the final wavefunction, performed through statistical sampling of $1.04 \times 10^6$ shots, revealed a high-dispersion regime with a Shannon entropy of $H \approx 15.71$ bits and an Inverse Participation Ratio (IPR) of $1.2 \times 10^{-4}$, with $261,661$ unique states observed. The absence of high IPR values suggests that the system does not exhibit localization ($\textit{"stability islands"}$), behaving predominantly as a robust entropy generator where information becomes extensively delocalized after wavefunction collapse.

In [2]:
%uv pip install --upgrade qiskit qiskit-aer numpy

Using Python 3.12.6 environment at: /usr/local
Resolved 11 packages in 74ms
Audited 11 packages in 0.11ms
Note: you may need to restart the kernel to use updated packages.

In [3]:
VECTOR_SIZE: int = 1024
CIRCUIT_QBITS: int = 17 # Twice the number of qubits you set will be used (e.g., 17 -> 34 qubits).

In [4]:
import logging

logging.basicConfig(level = logging.INFO, format = '[%(levelname)s:%(asctime)s] %(message)s')

In [5]:
from random import uniform

angle = lambda similary: similary * (pi - uniform(0.75, 0.075)) # Maps similarity to random quantum rotation angles.

In [6]:
from numpy import dot, array, kron, log2, sin, cos, pi, float32
from numpy.random import rand

from numpy.typing import NDArray

counter: int = 0
cache: dict[str, NDArray[float32]] = dict()

for _ in range(CIRCUIT_QBITS):

    # Initialize two vector of length `n`, in a scale from 0.0 to 1.0.
    vector_1: NDArray[float32] = rand(VECTOR_SIZE)
    vector_2: NDArray[float32] = rand(VECTOR_SIZE)
    
    similary: float = dot(vector_1, vector_2) # Dot product of the `vector_1` and `vector_2`.
    
    angle_value: float = angle(similary)
    
    logging.info(f"[{counter + 1}:{counter}] Angle: {angle_value:.4f} rad, Similarity: {similary:.4f}")

    # These define qubit amplitudes using the Bloch sphere angle (in radians).
    probability_one: float = sin(angle_value / 2)
    probability_zero: float = cos(angle_value / 2)

    # Create a two-qubit state vector from individual qubit amplitudes (create a two-dimensional Hilbert space).
    final_quantum_state: NDArray[float32] = array([probability_zero, probability_one], dtype = float32)

    logging.info(f"[{counter + 1}:{counter}] Final quantum state vector: {final_quantum_state}.")
    
    # Map interactions by combining dimensions from both matrices (v1 ⊗ v2).
    kron_product: NDArray[float32] = kron(final_quantum_state, final_quantum_state)

    # Normalize the `kron_product` vector to ensure total probability is one.
    norm: float = sum(x**2 for x in kron_product)**0.5
    kron_product = kron_product / norm
    
    logging.info(f"[{counter + 1}:{counter}] Kronecker product (normalized): {kron_product}.")

    cache [f"{counter}"]: NDArray[float32] = kron_product

    counter += 1

[INFO:2026-02-11 01:52:36,589] [1:0] Angle: 697.3514 rad, Similarity: 261.1182
[INFO:2026-02-11 01:52:36,591] [1:0] Final quantum state vector: [-0.9991568   0.04105731].
[INFO:2026-02-11 01:52:36,593] [1:0] Kronecker product (normalized): [ 0.99831426 -0.04102269 -0.04102269  0.0016857 ].
[INFO:2026-02-11 01:52:36,595] [2:1] Angle: 682.5654 rad, Similarity: 254.9323
[INFO:2026-02-11 01:52:36,596] [2:1] Final quantum state vector: [-0.40765214  0.9131373 ].
[INFO:2026-02-11 01:52:36,598] [2:1] Kronecker product (normalized): [ 0.16618027 -0.3722424  -0.3722424   0.83381975].
[INFO:2026-02-11 01:52:36,599] [3:2] Angle: 636.5467 rad, Similarity: 261.6836
[INFO:2026-02-11 01:52:36,601] [3:2] Final quantum state vector: [-0.56324995 -0.82628655].
[INFO:2026-02-11 01:52:36,602] [3:2] Kronecker product (normalized): [0.31725052 0.46540588 0.46540588 0.6827495 ].
[INFO:2026-02-11 01:52:36,603] [4:3] Angle: 698.2114 rad, Similarity: 261.8828
[INFO:2026-02-11 01:52:36,605] [4:3] Final quantum s

In [7]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.fake_provider import GenericBackendV2

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

"""
Configure a noisy simulation environment using 156-qubits.

num_qbuits -> Set the number of qubits for the backend simulation.
basis_gates -> Defines the native instructions the hardware can physically execute.
seed -> Random seed to reproduce the noisy environent.
"""
backend_boston_base = GenericBackendV2(
    num_qubits = 156, 
    basis_gates = ["cz", "id", "rx", "rz", "sx", "x"],
    seed = 42
)

"""
Use the Aer siulator to create a virtual quantum circuit.

method -> Uses a tensor-based method to simulate many entangled qubits.
device -> Specifies that the central processing unit runs the simulation.
max_parallel_threads -> Automatically uses all available processor cores for maximum speed.
noise_model -> Applies realistic hardware errors from the previously defined backend.
"""
aer_sim_boston = AerSimulator(
    method = "matrix_product_state",
    device = "CPU",
    max_parallel_threads = 0,
    noise_model = NoiseModel.from_backend(backend_boston_base)
)

qc = QuantumCircuit(CIRCUIT_QBITS * 2) # Set the used qbits in the circuit (e.g., 17 * 2 = 34 qubits).

for index, (key, vector) in enumerate(cache.items()):
    
    qubit_pair: list[int] = [index * 2, (index * 2) + 1]
    
    qc.initialize(vector, qubit_pair, normalize = True) # Sets the starting state of each qubit pair dynamically.
    
    qc.cx(qubit_pair[0], qubit_pair[1]) # Create entanglement between each qubit pair using CNOT gates.
    
qc.measure_all()

qc_transpiled = transpile(qc, aer_sim_boston, optimization_level = 3)

job = aer_sim_boston.run(qc_transpiled, shots = 1_048_576)
result = job.result()

counts: dict[str, int] = result.get_counts()

logging.info(f"{CIRCUIT_QBITS * 2} qbits used, results from Heron architecture (Boston): {counts}.")

[10+ MB] Logs stored on: /logs.txt

In [ ]:
from typing import Any

total_s: float = sum(counts.values())
probs: list[float] = [c/total_s for c in counts.values()]
entropy: float = -sum(p * log2(p) for p in probs)
max_ent: float = CIRCUIT_QBITS * 2
unique_states: int = len(counts)

ipr: float = sum(p**2 for p in probs)
ipr_normalized: float = ipr * len(counts)

results: dict[str, Any] = {
    "total_s": round(total_s, 6),
    "entropy": round(entropy, 6),
    "max_ent": round(max_ent, 6),
    "unique_states": unique_states,
    "ipr": ipr,
    "ipr_normalized": ipr_normalized
}

logging.info(results)

[INFO:2026-02-11 02:11:50,668] {'total_s': 1048576, 'entropy': np.float64(15.709585), 'max_ent': 34, 'unique_states': 261661, 'ipr': 0.00012473947936086915, 'ipr_normalized': 32.639456909044384}
